In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def weighted_rls_solution(weights:np.ndarray, X: np.ndarray, y:np.ndarray, lamb:float = 1.0) -> np.ndarray:
  """
    Inputs
      weights: one for each datapoint, shape (n,1) or (n,)
      X: design matrix shape (n, d)
      y: yvalues shape (n,1) or (n,)
      lamb: L2 regularization parameter, default 1.0
    Returns
      dx1 solution theta
  """
  n, d = X.shape
  weights = weights.reshape(n,-1)
  X_w = X * (weights**0.5)
  y_w = y.reshape(n, -1) * (weights**0.5)
  return np.linalg.solve(lamb * np.identity(d) + X_w.T @ X_w, X_w.T @ y_w)

def evaluate_weighted_rls_objective(theta: np.ndarray, weights:np.ndarray, X: np.ndarray, y:np.ndarray, lamb:float = 1.0) -> float:
  """
    Inputs
      theta: candidate vector for which we want to evaluate the objective value, shape (d, 1) or (d,)
      weights: one for each datapoint, shape (n,1) or (n,)
      X: design matrix shape (n, d)
      y: yvalues shape (n,1) or (n,)
      lamb: L2 regularization parameter, default 1.0
    Note: If you want to evaluate the objective on equal weighted rls objective just pass weights as 1/n...1/n
  """
  n, d = X.shape
  theta = theta.reshape(d, -1) # now shape (d, 1)
  weights = weights.reshape(n, -1) #now shape (n, 1)
  X_w = X * (weights**0.5) # shape (n, d)
  y_w = y.reshape(n, -1) * (weights**0.5) # shape (n, 1)
  return np.sum((X_w @ theta - y_w)**2) + lamb * np.linalg.norm(theta)**2

SAMPLING from L2 Laplace i.e. $p(x) \propto exp(-\beta ||x||_2)$

In [3]:
def sample_l2lap(beta:float, d:int) -> np.array:
  """
    Returns
      d dimensional noise sampled from `L2 laplace'
      https://math.stackexchange.com/questions/3801271/sampling-from-a-exponentiated-multivariate-distribution-with-l2-norm
  """
  R = np.random.gamma(d, scale = 1.0/beta)
  Z = np.random.normal(0, 1, size = d)
  return R * (Z / np.linalg.norm(Z))

Compute the beta used in L2 Laplace noise, this is then used to add central noise to the minimizer

In [4]:
def compute_beta(lamb, tot_epsilon):
  '''
    lamb: regularization parameter for ridge
    tot_epsilon: sum of all the agents privacy requirements \sum_i \varepsilon_i]
    Returns
      beta used for L2 laplace central noise
  '''
  return (lamb/2) * (lamb**0.5 / (1 + lamb**0.5)) * tot_epsilon

def compute_private_estimator(minimizer, beta, d):
  '''
    Private estimate after adding L2 laplace noise, note beta is calculated using epsilon required by each agent
  '''
  return minimizer + sample_l2lap(beta, d).reshape(d, -1)

NORMAL DIST -- SYNTHETIC DATASET

In [ ]:
def generate_linear_data(n:int, theta:np.array, sigma:float):
  '''
    Input:
      n: number of datapoints
      theta: the true theta used to generate the data, shape (d,), one dimensional array
      sigma: std for gaussian noise for the synthetic linear data
    returns
      X the design matrix shape is (n x d)
      y the associated labels shape is (n,)
  '''
  X = np.random.rand(n, len(theta))
  y = X @ theta + np.random.normal(0, sigma, size=n)
  return X, y

Generating the synthetic data

In [ ]:
from sklearn.preprocessing import normalize, minmax_scale

n = 100 #number of datapoints
theta = np.random.uniform(0, 10, size=10) # 3 dimensional
d = len(theta)
lamb = 1.0 # norm penalizer for ridge
X, y = generate_linear_data(n=n, theta=theta, sigma=0.1)

# Our theory is for all features within the L2 unit ball, and y's in [0,1]
X = normalize(X, norm='l2') # each row is L2 normalized
y = minmax_scale(y)

In [ ]:
# epsilons = np.random.uniform(1, 10, size=n) # epsilon required by each agent
epsilons = np.array([0.1]*(n//2) + [1]*(n//2))

### PERSONALIZED PRIVACY

In [ ]:
tot_epsilon = np.sum(epsilons)
weights_pp = epsilons/tot_epsilon # weights used in the ridge regression for personalized privacy

sol_exact_ridge_pp = weighted_rls_solution(weights_pp, X, y, lamb)
print("pluggin exact soln back into weighted ridge", evaluate_weighted_rls_objective(sol_exact_ridge_pp, weights_pp, X, y, lamb))
beta_pp = compute_beta(lamb, tot_epsilon)
print("beta for pp", beta_pp)
# to loop the part below
runs = 1000
unweighted_erm = []
weighted_erm = []
for _ in range(runs):
  theta_hat_pp = compute_private_estimator(sol_exact_ridge_pp, beta_pp , d)
  unweighted_erm.append(evaluate_weighted_rls_objective(theta_hat_pp, np.ones(n)/n, X, y, lamb))
  weighted_erm.append(evaluate_weighted_rls_objective(theta_hat_pp, weights_pp, X, y, lamb))
print("unweighted_erm_using_privateestimator", np.mean(unweighted_erm), np.std(unweighted_erm)) # WE care about low values here!
print("weighted_erm_using_privateestimator", np.mean(weighted_erm), np.std(weighted_erm))

### *Not* PERSONALIZED PRIVACY

  epsilon for all agents set to min of epsilons

In [ ]:
tot_epsilon = min(epsilons) * n
weights_npp = np.ones(n) / n

sol_exact_ridge_npp = weighted_rls_solution(weights_npp, X, y, lamb)
print("pluggin exact soln back into unweighted ridge", evaluate_weighted_rls_objective(sol_exact_ridge_npp, weights_npp, X, y, lamb))
beta_npp = compute_beta(lamb, tot_epsilon)
print("beta for not",beta_npp)
# to loop the part below
runs = 1000
unweighted_erm = []
weighted_erm = []
for _ in range(runs):
  theta_hat_npp = compute_private_estimator(sol_exact_ridge_npp, beta_npp , d)
  unweighted_erm.append(evaluate_weighted_rls_objective(theta_hat_npp, np.ones(n)/n, X, y, lamb))
  weighted_erm.append(evaluate_weighted_rls_objective(theta_hat_npp, weights_npp, X, y, lamb))
print("unweighted_erm_using_privateestimator", np.mean(unweighted_erm), np.std(unweighted_erm)) # WE care about low values here!
print("weighted_erm_using_privateestimator", np.mean(weighted_erm), np.std(weighted_erm))

# INSURANCE DATASET

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize, minmax_scale

def one_hot(df, cols): # idk if sklearns one-hot encoder is similar
    """
    df: pandas DataFrame
    param: cols a list of columns to encode 
    return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df
def numeric_scaler(df, cols):
    '''
    df: pandas dataframe
    numeric_cols: (array of strings) column names for numeric variables

    no return: does inplace operation
    '''
    df_new = df.copy()
    mmscaler = MinMaxScaler()
    df_new[cols] = mmscaler.fit_transform(df_new[cols])
    return df_new

In [6]:
df_medical = pd.read_csv('insurance.csv')

In [7]:
df_medical.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [8]:
numeric_all = ['age', 'bmi', 'children', 'charges']
cat_all = ['sex', 'smoker', 'region']

df_medical_mm = numeric_scaler(df_medical, numeric_all)
df_medical_mm_oh = one_hot(df_medical_mm, cat_all)
df_medical_mm_oh.drop(cat_all, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_medical_mm_oh = df_medical_mm_oh * 1.0 # make bool true, false into 1.0, 0.0

In [9]:
X = df_medical_mm_oh.drop('charges', axis=1).to_numpy()
y = df_medical_mm_oh['charges'].to_numpy()
X = normalize(X, norm='l2') # each row is L2 normalized
# y = minmax_scale(y)

In [11]:
n, d = X.shape
lamb = 1.0
epsilons = np.array([0.1]*(n//2) + [1]*(n//2)) # has to be even

### Personalized privacy

In [12]:
tot_epsilon = np.sum(epsilons)
weights_pp = epsilons/tot_epsilon # weights used in the ridge regression for personalized privacy

sol_exact_ridge_pp = weighted_rls_solution(weights_pp, X, y, lamb)
print("pluggin exact soln back into weighted ridge", evaluate_weighted_rls_objective(sol_exact_ridge_pp, weights_pp, X, y, lamb))
beta_pp = compute_beta(lamb, tot_epsilon)
print("beta for pp", beta_pp)
# to loop the part below
runs = 1000
unweighted_erm = []
weighted_erm = []
for _ in range(runs):
  theta_hat_pp = compute_private_estimator(sol_exact_ridge_pp, beta_pp , d)
  unweighted_erm.append(evaluate_weighted_rls_objective(theta_hat_pp, np.ones(n)/n, X, y, lamb))
  weighted_erm.append(evaluate_weighted_rls_objective(theta_hat_pp, weights_pp, X, y, lamb))
print("unweighted_erm_using_privateestimator", np.mean(unweighted_erm), np.std(unweighted_erm)) # WE care about low values here!
print("weighted_erm_using_privateestimator", np.mean(weighted_erm), np.std(weighted_erm))

pluggin exact soln back into weighted ridge 0.06143817955856497
beta for pp 183.975
unweighted_erm_using_privateestimator 0.06647168255692947 0.002625592123434555
weighted_erm_using_privateestimator 0.06565153591387154 0.002621507342250632


### Not personalized privacy

In [14]:
tot_epsilon = min(epsilons) * n
weights_npp = np.ones(n) / n

sol_exact_ridge_npp = weighted_rls_solution(weights_npp, X, y, lamb)
print("pluggin exact soln back into unweighted ridge", evaluate_weighted_rls_objective(sol_exact_ridge_npp, weights_npp, X, y, lamb))
beta_npp = compute_beta(lamb, tot_epsilon)
print("beta for not",beta_npp)
# to loop the part below
runs = 1000
unweighted_erm = []
weighted_erm = []
for _ in range(runs):
  theta_hat_npp = compute_private_estimator(sol_exact_ridge_npp, beta_npp , d)
  unweighted_erm.append(evaluate_weighted_rls_objective(theta_hat_npp, np.ones(n)/n, X, y, lamb))
  weighted_erm.append(evaluate_weighted_rls_objective(theta_hat_npp, weights_npp, X, y, lamb))
print("unweighted_erm_using_privateestimator", np.mean(unweighted_erm), np.std(unweighted_erm)) # WE care about low values here!
print("weighted_erm_using_privateestimator", np.mean(weighted_erm), np.std(weighted_erm))

pluggin exact soln back into unweighted ridge 0.06224927666537697
beta for not 33.45
unweighted_erm_using_privateestimator 0.18742785953975424 0.07416783950719678
weighted_erm_using_privateestimator 0.18742785953975424 0.07416783950719678
